# Tutorial of merging LiDAR data and extracting LiDAR features for WOMD

This tutorial demonstrates how to add LiDAR data to the original WOMD scenes. It also provides methods to extract LiDAR points and features from the merged scenario proto message.

## Install

To run Jupyter Notebook locally:

```
python3 -m pip install waymo-open-dataset-tf-2-11-0==1.5.2
python3 -m pip install "notebook>=5.3" "ipywidgets>=7.5"
python3 -m pip install --upgrade jupyter_http_over_ws>=0.0.7 && \
jupyter serverextension enable --py jupyter_http_over_ws
jupyter notebook
```

In [ ]:
import os
from typing import List, Dict, Tuple, Optional, Any

import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

from waymo_open_dataset import dataset_pb2
from waymo_open_dataset.protos import scenario_pb2
from waymo_open_dataset.protos import compressed_lidar_pb2
from waymo_open_dataset.utils import womd_lidar_utils

# Augmenting a WOMD scenario

To augment the original WOMD with laser data for input frames, there are three steps:
1. Load a scenario proto message, and check the `scenario_id` field.
2. Find the corresponding compressed frame laser data file which has the file name as `{scenario_id}.tfrecord`.
3. Load the frame laser data file which is a scenario proto with non-empty `compressed_frame_laser_data` field only and merge the loaded data into the scenario proto's `compressed_frame_laser_data` field.

In [ ]:
def _load_scenario_data(tfrecord_file: str) -> scenario_pb2.Scenario:
  """Load a scenario proto from a tfrecord dataset file."""
  dataset = tf.data.TFRecordDataset(tfrecord_file, compression_type='')
  data = next(iter(dataset))
  return scenario_pb2.Scenario.FromString(data.numpy())

WOMD_FILE = '/content/waymo-od/tutorial/womd_scenario_input.tfrecord'
womd_original_scenario = _load_scenario_data(WOMD_FILE)
print(womd_original_scenario.scenario_id)

In [ ]:
# The corresponding compressed laser data file has the name
# {scenario_id}.tfrecord. For simplicity, we rename the corresponding laser data
# file 'ee519cf571686d19.tfrecord' to be 'womd_lidar_data.tfrecord'.
LIDAR_DATA_FILE = '/content/waymo-od/tutorial/womd_lidar_data.tfrecord'
womd_lidar_scenario = _load_scenario_data(LIDAR_DATA_FILE)
scenario_augmented = womd_lidar_utils.augment_womd_scenario_with_lidar_points(
    womd_original_scenario, womd_lidar_scenario)
print(len(scenario_augmented.compressed_frame_laser_data))

# Extract the lidar point clouds

In [ ]:
frame_points_xyz = {}  # map from frame indices to point clouds
frame_points_feature = {}
frame_i = 0

def _get_laser_calib(
    frame_lasers: compressed_lidar_pb2.CompressedFrameLaserData,
    laser_name: dataset_pb2.LaserName.Name):
  for laser_calib in frame_lasers.laser_calibrations:
    if laser_calib.name == laser_name:
      return laser_calib
  return None

# Extract point cloud xyz and features from each LiDAR and merge them for each
# laser frame in the scenario proto.
for frame_lasers in scenario_augmented.compressed_frame_laser_data:
  points_xyz_list = []
  points_feature_list = []
  frame_pose = np.reshape(np.array(
      scenario_augmented.compressed_frame_laser_data[frame_i].pose.transform),
      (4, 4))
  for laser in frame_lasers.lasers:
    if laser.name == dataset_pb2.LaserName.TOP:
      c = _get_laser_calib(frame_lasers, laser.name)
      (points_xyz, points_feature,
       points_xyz_return2,
       points_feature_return2) = womd_lidar_utils.extract_top_lidar_points(
           laser, frame_pose, c)
    else:
      c = _get_laser_calib(frame_lasers, laser.name)
      (points_xyz, points_feature,
       points_xyz_return2,
       points_feature_return2) = womd_lidar_utils.extract_side_lidar_points(
           laser, c)
    points_xyz_list.append(points_xyz.numpy())
    points_xyz_list.append(points_xyz_return2.numpy())
    points_feature_list.append(points_feature.numpy())
    points_feature_list.append(points_feature_return2.numpy())
  frame_points_xyz[frame_i] = np.concatenate(points_xyz_list, axis=0)
  frame_points_feature[frame_i] = np.concatenate(points_feature_list, axis=0)
  frame_i += 1

In [ ]:
print(frame_points_xyz[0].shape)
print(frame_points_feature[0].shape)

###Show point cloud
3D point clouds are rendered using an internal tool, which is unfortunately not publicly available yet. Here is an example of what they look like.

In [ ]:
from IPython.display import Image, display
display(Image('/content/waymo-od/tutorial/womd_point_cloud.png'))